#Module 1b: Aggregating pyWaPOR outputs into dekadal/ monthly and seasonal maps

In this Notebook we use a script that was developed for the WaPOR project to calculate temporal aggregation of pyWaPOR output (AETI, T and NPP), which are needed to run the irrigation performance assessment module.
This script exports the WaPOR netCDF file into dekadal/ monthly and seasonal TIFF maps.

This script follows the following steps:

*  Step 0: Import modules/libraries
*  Step 1: Define functions for aggregating pyWaPOR outputs
*  Step 2: Read pywapor output and reproject the xarray
*  Step 3: Aggregate to the required timestep
*  Step 4: Write the result



## Step 0. Import modules/libraries

In [ ]:
%%capture
!pip install --upgrade xarray --quiet
!pip install --upgrade rioxarray --quiet


In [ ]:
import xarray as xr
import rioxarray as rio
from rasterio.warp import reproject, Resampling
import pandas as pd
import numpy as np
import datetime
import os

## Step 1: Define function to aggregate pyWaPOR output data

In [ ]:
# functions to calculate dekadal, monthly and seasonal sums from the ET_look output netcdf file

# decadal sum
def dekadal_sum(ds):
    # Store encoding information
    encoding = {var: ds[var].encoding for var in ds.data_vars}

    d = ds.time.dt.day - np.clip((ds.time.dt.day-1) // 10, 0, 2)*10 - 1
    date = ds.time.values - np.array(d, dtype="timedelta64[D]")
    ds['time'] = date
    ds_dk = ds.groupby(ds.time).sum(dim='time', keep_attrs=True)
    # Restore encoding information
    for var in ds_dk.data_vars:
        if var in encoding:  # Ensure the variable exists in the original encoding
            ds_dk[var].encoding = encoding[var]

    return ds_dk

# Monthly sum
def monthly_sum(ds):
    # Store encoding information
    encoding = {var: ds[var].encoding for var in ds.data_vars}

    ds_mn = ds.resample(time="1ME").sum()
    # Restore encoding information
    for var in ds_mn.data_vars:
        if var in encoding:  # Ensure the variable exists in the original encoding
            ds_mn[var].encoding = encoding[var]
    return ds_mn
# Check if the start and end time of the selected dataarray corresponds to sos and eos, if not adjust the data array based on the times
def select_season_da(da_var, season_start_date, season_end_date):

    sos = datetime.datetime.fromisoformat(season_start_date) #start of season date, we use datetime.datetime to convert the year, month, day to a datetime object
    eos = datetime.datetime.fromisoformat(season_end_date) #end of season date
    da = da_var.sel(time=slice(sos, eos))
    st_da = da.time[0]
    et_da = da.time[-1]

    st_dif = pd.to_datetime(st_da.data) - sos
    en_dif = eos - pd.to_datetime(et_da.data)

    idx1 = list(da_var.time.values).index(da_var.sel(time=da.time[0], method='nearest').time)
    idx2 = list(da_var.time.values).index(da_var.sel(time=da.time[-1], method='nearest').time)

    d1 = st_dif.days
    d2 = (pd.to_datetime(da.time[0].data)-pd.to_datetime(da_var.time[idx1-1].data)).days

    da_fr1 = da_var.sel(time=da_var.time[idx1-1])*d1/d2
    da_fr1['time'] = sos

    d3 = en_dif.days
    d4 = (pd.to_datetime(da_var.time[idx2+1].data) - pd.to_datetime(da.time[-1].data)).days

    da_fr2 = da_var.sel(time=da_var.time[idx1+1])*d3/d4
    da_fr2['time'] = eos

    if da_fr1.time.size != 0:
        da = xr.concat([da_fr1, da], dim = 'time')

    if da_fr2.time.size != 0:
        da = xr.concat([da, da_fr2], dim = 'time')

    return da

# Seasonal Resample
def seasonal_sum(ds, sos, eos):
    # Store encoding information
    encoding = {var: ds[var].encoding for var in ds.data_vars}

    ds_sn = select_season_da(ds, sos, eos)
    ds_sn = ds_sn.sum(dim = 'time')
    # Restore encoding information
    for var in ds_sn.data_vars:
        if var in encoding:  # Ensure the variable exists in the original encoding
            ds_sn[var].encoding = encoding[var]
    return ds_sn

# reproject the dataset
def reproject(ds):
  crs_info = input(f"The estimated UTM crs is {ds.rio.estimate_utm_crs()}.\nWould like to repoject the dataset? Enter a valid EPSG code\
  or a template raster file path: ")
  try:
      # Convert it into integer
      to_crs = int(crs_info)
      print(f"project to EPSG:{crs_info}")
      to_crs = f"EPSG:{to_crs}"
      dst = ds.rio.reproject(to_crs, resampling=Resampling.nearest, nodata=-9999)
      return dst
  except ValueError:
      try:
          if os.path.exists(crs_info):
              print("Use a template raster to repoject the dataset")
              temp_rst_file = crs_info
              da_rst = rio.open_rasterio(temp_rst_file)
              if da_rst.rio.crs != None:
                  dst= ds.rio.reproject_match(da_rst, nodata=-9999)
                  return dst
              else:
                  print(f"the template raster {temp_rst_file} does not have CRS information.")
      except ValueError:
          print("Your input is not either a valid EPSG code or a teplate raster path.")

# netCDF to geotiff
def write2gtiff(ds, temporal_res, dir_out):

  date_str = pd.to_datetime(ds.time.data).strftime('%Y-%m-%d')
  # Create folder per variable.
  dir_out = r'/content/'+dir_out
  if not os.path.isdir(dir_out):
      os.makedirs(dir_out)

  for var in ds.data_vars:
    var_name = var.split('_')[0]
    fd = os.path.join(dir_out, temporal_res, var_name)

    encoding  = ds[var].encoding
    attrs = ds[var].attrs
    # Create folder per variable.
    if not os.path.isdir(fd):
        os.makedirs(fd)

    for i in range(len(ds.time)):
      fname = os.path.join(fd, f"{var_name}_{date_str[i]}")
      da = ds[var][i]
      da = da.drop_vars('time')  # get the data for one time step
      da = da.where(da==encoding['_FillValue'], da*encoding['scale_factor'])

      # Modify the attributes

      attrs.update({'date': date_str[i],
                    'units' : f"mm/{temporal_res}",
                    'temporal_resolution' : temporal_res,
                    'scale_factor' :1})
      attrs_to_delete = [i for i in attrs if 'NETCDF_' in i]
      attrs_new = {key: attrs[key] for key in attrs if key not in attrs_to_delete}
      da.attrs  = attrs_new
      # print(i, attrs_new)
      # write the dataarray to a geotif file
      da.rio.to_raster(f"{fname}.tif", driver="GTiff", compress="LZW")
      da.close()

## Step 2: Read pywapor output and reproject the xarray

Set the path to the ET Look output file.

In [ ]:
# path to the et_look_out/nc file
path_et_look_out = r'/content/et_look_out.nc'
ds = xr.open_dataset(path_et_look_out, decode_coords="all")
ds = ds.rename({'time_bins': 'time'})
# ds


 Reproject the xarray dataset if needed
The ET Look output is in EPSG:4326, use the next cell to reproject the dataset to other projections, otherwise skip it.

In [ ]:
dst = reproject(ds)


## Step 3: Aggregate to the required timestep (dekadal, monthly or seasonal)

In [ ]:
# aggregate to the required timestep
ds_mn = monthly_sum(dst) # monthly
ds_dk = dekadal_sum(dst) # dekadal

# for seasonal aggregation use the lines below:
# aggregate to seasonal timestep (if applicable)
# season_start_date = '2022-10-05' # start of the season in iso format
# season_end_date = '2023-02-28' # end of the season in iso format
# ds_sn = seasonal_sum(ds, season_start_date, season_end_date)

## Step 4: Write the result to individual TIFF files per time step

In [ ]:

# create directory to store the tiff files
dir_out = r'output' # folder to save the dekadal geotif files
temporal_res = 'dekadal'
write2gtiff(ds_dk, temporal_res, dir_out)

#dir_out = r'output' # folder to save the monthly geotif files
#temporal_res = 'monthly'
#write2gtiff(ds_mn, temporal_res, dir_out)

#dir_out = r'output' # folder to save the seasonal geotif files
#temporal_res = 'seasonal'
#write2gtiff(ds_sn, temporal_res, dir_out)

In [ ]:
# # test opening one of the geotiff files
# rst_path = r'/content/output/dekadal/e/e_2022-10-01.tif'
# da = rio.open_rasterio(rst_path)
# da.plot()
# da.close()

In [ ]:
!zip -r /content/pywapor_out.zip /content/output
from google.colab import files
files.download(r'/content/pywapor_out.zip')